# Reaction Center Prediction - Pilot Demo

Demonstration of rule-based reaction center prediction for polyol/sugar oxidation and isomerization reactions.

In [ ]:
import sys
sys.path.append('../src')

from rules.rule_based_predictor import RuleBasedPredictor
from evaluation.evaluator import ReactionCenterEvaluator, TestSample, BaselinePredictor
from rdkit import Chem
from rdkit.Chem import Draw
import pandas as pd

## 1. Test Molecules

Common polyols and sugars used in biosynthesis

In [ ]:
test_molecules = [
    ("OCC(O)C(O)C(O)C(O)CO", "D-sorbitol"),
    ("OCC(=O)C(O)C(O)C(O)CO", "D-fructose"),
    ("OCC(O)C(O)C(O)CO", "xylitol"),
    ("OCC(=O)C(O)C(O)CO", "D-xylulose"),
    ("OCC(O)C(O)C(O)C(O)C=O", "D-glucose"),
]

mols = [(Chem.MolFromSmiles(smiles), name) for smiles, name in test_molecules]
img = Draw.MolsToGridImage([m[0] for m in mols], legends=[m[1] for m in mols], molsPerRow=3)
img

## 2. Rule-Based Prediction

Apply chemical transformation rules to identify reaction centers

In [ ]:
predictor = RuleBasedPredictor()

print("Available Rules:")
for i, rule in enumerate(predictor.rules, 1):
    print(f"{i}. {rule.name} (EC {rule.ec_class})")
    print(f"   Substrate: {rule.smarts_substrate}")
    print(f"   Priority: {rule.priority}\n")

## 3. Predict Oxidation Sites

In [ ]:
results = []

for smiles, name in test_molecules:
    print(f"\n{'='*60}")
    print(f"{name}: {smiles}")
    print('='*60)
    
    for reaction_type in ["oxidation", "isomerization"]:
        predictions = predictor.predict_reaction_centers(smiles, reaction_type)
        
        print(f"\n{reaction_type.upper()}:")
        
        if predictions:
            for i, pred in enumerate(predictions[:3], 1):
                print(f"  {i}. Atoms {pred.atom_indices}")
                print(f"     Rule: {pred.rule_name}")
                print(f"     Confidence: {pred.confidence:.3f}")
                print(f"     EC: {pred.ec_class}, Cofactor: {pred.cofactor}")
                
                results.append({
                    'molecule': name,
                    'reaction_type': reaction_type,
                    'rank': i,
                    'atoms': str(pred.atom_indices),
                    'rule': pred.rule_name,
                    'confidence': pred.confidence,
                    'ec_class': pred.ec_class
                })
        else:
            print("  No predictions")

## 4. Results Summary

In [ ]:
df = pd.DataFrame(results)
df

## 5. Visualize Predicted Reaction Centers

Highlight predicted atoms on molecular structures

In [ ]:
from rdkit.Chem import AllChem
from rdkit.Chem.Draw import rdMolDraw2D
from IPython.display import SVG

def visualize_reaction_center(smiles, atom_indices, title=""):
    mol = Chem.MolFromSmiles(smiles)
    
    drawer = rdMolDraw2D.MolDraw2DSVG(400, 300)
    drawer.DrawMolecule(mol, highlightAtoms=atom_indices)
    drawer.FinishDrawing()
    
    svg = drawer.GetDrawingText()
    return SVG(svg)

smiles, name = test_molecules[0]
predictions = predictor.predict_reaction_centers(smiles, "oxidation")

if predictions:
    print(f"Top prediction for {name} oxidation:")
    print(f"Atoms: {predictions[0].atom_indices}")
    print(f"Rule: {predictions[0].rule_name}")
    visualize_reaction_center(smiles, predictions[0].atom_indices, name)

## 6. Evaluation Framework Demo

Demonstrate evaluation metrics with synthetic test data

In [ ]:
test_samples = [
    TestSample(
        substrate_smiles="OCC(O)C(O)C(O)C(O)CO",
        true_reaction_center=[2],
        reaction_type="oxidation",
        ec_number="1.1.1.14",
        reaction_id="sorbitol_to_fructose"
    ),
    TestSample(
        substrate_smiles="OCC(O)C(O)C(O)CO",
        true_reaction_center=[2],
        reaction_type="oxidation",
        ec_number="1.1.1.9",
        reaction_id="xylitol_to_xylulose"
    ),
]

evaluator = ReactionCenterEvaluator(test_samples)

print("Evaluating Rule-Based Predictor...\n")
result = evaluator.evaluate(predictor)
evaluator.print_report(result)

## 7. Baseline Comparison

In [ ]:
baseline = BaselinePredictor()

print("Evaluating Random Baseline...\n")
baseline_result = evaluator.evaluate(baseline)
evaluator.print_report(baseline_result)

print("\n" + "="*60)
print("COMPARISON")
print("="*60)
print(f"\nTop-5 Accuracy:")
print(f"  Random Baseline: {baseline_result.top5_accuracy:.1f}%")
print(f"  Rule-Based: {result.top5_accuracy:.1f}%")
print(f"  Improvement: {result.top5_accuracy - baseline_result.top5_accuracy:.1f}%")

## 8. Next Steps

### Phase 1 Complete ✓
- Rule-based baseline implemented
- Evaluation framework established
- Practical impact metrics defined

### Phase 2: Data & ML
1. Extract real reactions from Rhea database
2. Build labeled training dataset
3. Train simple ML model (gradient boosting)
4. Compare ML vs rules

### Phase 3: Expansion
1. Add more EC classes
2. Incorporate enzyme specificity data
3. Build enzyme recommendation system